
Notebook originally taken from (but modified here):  
https://github.com/daydrill/ga_pycon_2016_apac

Slides taken from:  
https://github.com/pythonkr/pyconapac-2016-files/raw/master/20160814-102-16-SongChisung.pdf




## Brief Introduction

### Genetics and genetic programming

![song01](images/song01.png)

Population, chromosomes and genes:  
![song01](images/song02.png)

Basic genetic programming operations:  
![song01](images/song03.png)

Selection:  
![song01](images/song04.png)

Crossover (gene sharing):  
![song01](images/song05.png)

Mutation (gene modification):  
![song01](images/song06.png)

GA/GP workflow:  
![song01](images/song07.png)


### DEAP Architecture


![song01](images/song08.png)

![song01](images/song09.png)


<hr style="border-color:#ff9900"> 

## Practice 1 : 

<hr style="border-color:#ff9900"> 

This example uses DEAP to create a list of numbers that add up to some value.

## Setting Things Up


Assuming that you alreadyhave DEAP type the following on your shell:

    conda install -c conda-forge folium 



In [1]:
import random
import numpy as np
from deap import algorithms, base, creator, tools

doVerbose = True
listLength = 5
sumUpTo = 100

In [2]:
if doVerbose:
    # manually compile list:
    # Here are 5 numbers when added, we get a hundred.
    list1 = [100, 0, 0, 0, 0]
    list2 = [20, 21, 19, 15, 25]
    (sum(list1), sum(list2))

### Creator
- Meta-factory allowing the run-time creation of classes via both inheritance and composition.
- Attributes, both data and functions, can be dynamically added to existing classes in order to create user-specific new types.
- By using this, the creation of individuals and populations from any data structure ( list, set, dictionary, tree, etc… )

In [3]:
# Creates a new class named "FitnessMin" inheriting from "base.Fitness" with attribute "weights=(-1.0,)"
# The fitness is a measure of quality of a solution.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # -1 -> minimum problem
creator.create("Individual", list, fitness=creator.FitnessMin)

### Toolbox
- Container for the tools (operators) that the user wants to use.
- Manually populated by the user with selected tools.

In [4]:
toolbox = base.Toolbox()

# Attribute generator : get random number between 0~100
toolbox.register("attr_bool", random.randint, 0, sumUpTo)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, listLength)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


## The Evaluation Function

objective function  built for our problem: want to minimise the sum distance from `sumUpTo`

In [5]:
def sum_error(individual):
    error = abs(sumUpTo - sum(individual))
    # note the ',' and end of line, makes a tuple:
    return error,

In [6]:
if doVerbose:
    ind = toolbox.individual()
    print('Demonstrate the attribute, individual and population:')
    print(f'Attribute demo: {toolbox.attr_bool()}') 
    print(f'Individual demo: {ind}')  
    print(f'Individual sum: {sum(ind)}')  # demo test individual sum
    print(type(sum_error(ind))) # error
    print(sum_error(ind)) # error
    print(toolbox.population(n=10)) # demo pop of 10 indv
    

Demonstrate the attribute, individual and population:
Attribute demo: 86
Individual demo: [51, 66, 70, 35, 5]
Individual sum: 227
<class 'tuple'>
(127,)
[[83, 61, 66, 36, 4], [33, 100, 14, 81, 68], [73, 99, 80, 18, 73], [7, 100, 48, 74, 30], [29, 94, 18, 35, 23], [12, 86, 73, 27, 46], [49, 45, 87, 69, 54], [71, 4, 83, 23, 8], [6, 100, 69, 84, 33], [93, 99, 51, 43, 13]]


## The Genetic Operators
- Operators are just like initializers, except that some are already implemented in the [tools](http://deap.readthedocs.io/en/master/api/tools.html#module-deap.tools) module. 
- __Once you’ve chosen the perfect ones, simply register them in the toolbox.__

In [7]:
toolbox.register("evaluate", sum_error)

In [8]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.2) # Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("select", tools.selTournament, tournsize=3)

## Evolving the Population

### Creating the Population

In [9]:
pop = toolbox.population(n=100)
pop[:10] # only print the first few

[[12, 47, 55, 37, 76],
 [97, 72, 20, 69, 31],
 [76, 22, 10, 26, 22],
 [51, 59, 94, 61, 45],
 [43, 0, 79, 50, 92],
 [96, 83, 68, 43, 74],
 [47, 86, 65, 50, 26],
 [24, 14, 95, 98, 78],
 [31, 90, 83, 5, 26],
 [13, 89, 38, 42, 14]]

### The Appeal of Evolution

In [10]:
# Use of a HallOfFame in order to keep track of the best individual to appear in the evolution 
# (it keeps it even in the case it extinguishes)
hof = tools.HallOfFame(1)

In [11]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [12]:
# algorithms : contains useful implements some basic GA
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)
print(log)
print(pop)

gen	nevals	avg   	std    	min	max
0  	100   	158.96	68.0648	16 	304
1  	59    	96.09 	46.4719	2  	197
2  	64    	60.57 	43.0075	1  	210
3  	66    	34.44 	26.1198	1  	132
4  	67    	25.62 	22.0494	1  	116
5  	58    	19.64 	24.1062	0  	141
6  	49    	17.44 	22.5895	0  	107
7  	66    	18.49 	27.9075	0  	168
8  	58    	15.45 	25.1906	0  	161
9  	64    	14.77 	26.0948	0  	158
10 	71    	12.29 	17.118 	0  	105
11 	60    	11.02 	24.4896	0  	135
12 	46    	7.73  	19.021 	0  	100
13 	52    	8.76  	26.3678	0  	162
14 	63    	4.65  	14.4356	0  	90 
15 	61    	7.92  	21.5632	0  	99 
16 	73    	13.43 	29.5568	0  	135
17 	52    	4.3   	15.0542	0  	92 
18 	55    	3.96  	13.0376	0  	85 
19 	64    	9.21  	25.5054	0  	119
20 	57    	6.52  	19.5318	0  	88 
21 	69    	7.24  	20.4015	0  	95 
22 	61    	8.98  	28.7367	0  	201
23 	52    	7.77  	21.5076	0  	89 
24 	62    	8.45  	29.702 	0  	178
25 	55    	6.14  	20.0709	0  	94 
26 	58    	7.74  	20.4003	0  	95 
27 	62    	9.16  	24.4862	0  	140
28 	57    	12.

In [13]:
[sum(ind) for ind in pop[:10]] # first ten

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

In [14]:
tools.selBest(pop, k=5)

[[1, 21, 10, 16, 52],
 [1, 21, 10, 16, 52],
 [1, 21, 10, 16, 52],
 [1, 21, 10, 16, 52],
 [1, 21, 10, 16, 52]]